<a href="https://colab.research.google.com/github/OlegPozovnoy/MailRuMLHomeworks/blob/master/HW3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/datasets/')

Drive already mounted at /datasets/; to attempt to forcibly remount, call drive.mount("/datasets/", force_remount=True).


In [2]:
!ls "/datasets/My Drive/datasets/"

players.pkl  results.pkl  tournaments.pkl


In [0]:
import pickle
import pandas as pd

In [0]:
tournaments = pd.read_pickle("/datasets/My Drive/datasets/tournaments.pkl")
#tournaments = pd.read_pickle("./tournaments.pkl")

tournaments_cleaned = [(v['id'], v['name'], v['dateStart'][0:4], v['type']['name'], v['questionQty']) for k,v in tournaments.items() if v['questionQty'] != None]
del tournaments

Создадим трейн и тест набор турниров.

In [5]:
train_id = [ v[0] for v in tournaments_cleaned if int(v[2])== 2019]
test_id = [v[0] for v in tournaments_cleaned if int(v[2])== 2020]

print(len(train_id))
print(len(test_id))

687
418


In [0]:
results = pd.read_pickle("/datasets/My Drive/datasets/results.pkl")
#results = pd.read_pickle("./results.pkl")
results_cleaned = {k: v for k,v in results.items() if k in train_id or k in test_id}

results_cleaned2 = {}
for k,v in results_cleaned.items():
  results_cleaned2[k] = []
  for s in v:
    if s.get('mask') != None:
      results_cleaned2[k].append((s.get('mask'),s.get('questionsTotal'), [k.get('player').get('id') for k in s.get('teamMembers')]))

del results
del results_cleaned

In [7]:
train_id = [id for id in train_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]
test_id = [id for id in test_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]

print(len(train_id))
print(len(test_id))

675
173


In [8]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import tqdm
import gc

# creating all questions and players set
all_words = []
for k,v in results_cleaned2.items():
  for team in v: 
    for p in range(len(team[0])):
        all_words.append(str(k)+'_'+str(p))
    for m in team[2]:
        all_words.append(m)

all_words = set(all_words)
print(len(all_words))

# creating storage for competetion results
all_results = {}
for s in all_words:
  all_results[s] = [0,0] # total rating and num of competetions


logistic = LogisticRegression(fit_intercept = False)

def process_tournament(id):
  task3 = []
  words = []
   
  for n in range(len(results_cleaned2[id])): # command id
    for p in range(len(results_cleaned2[id][n][0])): #answers
        quest = str(id)+'_'+str(p)
        team = n
        task3.append((quest, team,results_cleaned2[id][n][0][p]) )
        words.append(quest)
        words.append(team)

  if len(words)==0:
    return 
  
  words = set(words) # encoding questions and teams no

  idx_to_word = dict(enumerate(words))
  word_to_idx = dict([(y,x) for x,y in idx_to_word.items()])
  #preparing matrix
  X = np.zeros((len(task3),len(words)))
  Y = np.zeros(len(task3))
  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0

  if len(set(Y)) <= 1:
    print ('<2 categories , skipping')
    return

  #print(X.shape)
  logistic.fit(X, Y)

  for s in range(len(logistic.coef_[0])):
    if isinstance(idx_to_word[s], int): ## teanm no
        #print('team '+ str(idx_to_word[s]))
        for player in results_cleaned2[id][idx_to_word[s]][2]:
            #print(player)
            all_results[player][1] +=1
            all_results[player][0] +=logistic.coef_[0][s]
    else: #question no
        #print('question '+ str(idx_to_word[s]))
        all_results[idx_to_word[s]][1] +=1
        all_results[idx_to_word[s]][0] +=logistic.coef_[0][s]

#process_tournament(1526)


104707


Пройдемся по всем турнирам и посчитаем рейтинг всех игроков и вопросов в заданном турнире, за неимением лучшего, усредним по каждому игроку рейтинг в турнире

In [9]:
import tqdm

for tourn_id in tqdm.tqdm(train_id):
  process_tournament(tourn_id)

 99%|█████████▉| 670/675 [01:56<00:00, 28.45it/s]

<2 categories , skipping


100%|██████████| 675/675 [02:47<00:00,  4.03it/s]


In [10]:
import statistics as stat

def is_player(t):
  return isinstance(t, int)

all_ranking = {}

for k,v in all_results.items():
  if v[1] != 0:
    all_ranking[k] = v[0]/v[1] 

blank_rank = stat.mean ([ v for k,v in all_ranking.items() if is_player(k)])
print(blank_rank)


-0.5379797641338613


In [0]:
def get_team_ranking(players):
  result = []
  print('Team:',players)
  if (len(players)==0):
    return blank_rank
  for player in players:
    if (all_results[player][1]>0): # num of competetions
      #print(player, ' rating is ', all_ranking[player])
      result.append(all_ranking[player])
    else:
      result.append(blank_rank)
  return stat.mean(result)


In [12]:
from scipy import stats

spearman = []
pearson = []

test_id = [x for x in test_id if len(results_cleaned2[x])>1] ## фильтруем турниры с 1 командой

for tourn_id in test_id:
  print('Tournament:', tourn_id)
  rating = []
  ranking = []
  i=1
  for team in results_cleaned2[tourn_id]:
    rating.append(get_team_ranking(team[2]))
    ranking.append(-i)
    i+=1
  #print ('Spearman: ',stats.spearmanr(rating, ranking)[0])
  #print ('Pearson: ',stats.pearsonr(rating,ranking)[0])
  # ставим минус т.к. у нас места возрастают а сила убывает
  spearman.append(stats.spearmanr(rating, ranking)[0])
  pearson.append(stats.pearsonr(rating,ranking)[0])

print(spearman)
print(pearson)
print(stat.mean(spearman))
print(stat.mean(pearson))


Streaming output truncated to the last 5000 lines.
Team: [180448, 180449, 197802, 197803, 200990, 200991]
Team: [217110, 217112, 219489, 217111, 217109, 218945]
Team: [135728, 160350, 214301, 214300, 214302, 219090]
Tournament: 6185
Team: [9342, 77673, 10580, 71907, 38307, 43559]
Team: [21026, 15123, 1354, 96642, 66401, 118142]
Team: [126620, 23967, 147750, 28019, 10083, 73380]
Team: [5007, 7737, 29487, 39152, 27436, 24609]
Team: [29578, 6076, 17955, 39879, 173665, 10387]
Team: [3088, 52047, 23486, 1599, 36673, 24844]
Team: [30772, 34417, 52183, 93424, 29425, 13345]
Team: [30371, 61316, 149363, 86251, 103351, 66180]
Team: [1369, 28426, 16703, 7669, 38440, 11491]
Team: [38180, 17780, 13196, 30014, 23996, 27773]
Team: [158734, 3440, 114127, 121186, 112086, 36086]
Team: [26477, 25110, 34684, 34682, 27245, 162306]
Team: [103745, 33706, 22208, 194371, 194328, 195615]
Team: [150165, 137194, 15726, 134619, 208796, 16129]
Team: [102213, 150355, 195793, 39503, 93018, 102548]
Team: [10474, 23804

Теперь будем работать в рамках модели, когда участники команды отвечают на вопрос независимо и если кто то ответил - команда отвечает правильно, если никто - не отвечает. Будем попеременно улучшать оценки для сложностей вопросов и индивидуальных показателей всех участников

In [0]:
from scipy.special import expit
import math

def sigmoid(x):
  return expit(x)

def sigm_reverse(x):
  if x<0.001:
    return -999
  elif x>0.999:
    return 999
  return -math.log(1/x-1)

def adjust_individual(players, team_X, Y):
  print('Adjust individual rank by team rank: ', players, team_X, Y)
  prob_real = 1-sigmoid(team_X + Y)
  if len(players) ==0:
    return
  #print('Prob_real:',prob_real)
  prob_adjusted = 1
  probs = []
  player_ratings = []
  for player in players:
    player_rating = all_ranking[player]
    player_ratings.append(player_rating)
    player_prob = (1 - sigmoid(player_rating + Y))
    probs.append(player_prob)
    prob_adjusted *= (1 - sigmoid(player_rating + Y))
  #print('Prob_adjusted', prob_adjusted)
  multiplier = math.pow(prob_real/prob_adjusted, 1.0/len(players))
  #print(multiplier)
  probs = [x*multiplier for x in probs]
  #print(probs)
  #print([sigm_reverse(1-x) - Y for x in probs])
  result = [sigm_reverse(1-x) - Y for x in probs] 
  print ('Before adjustment: ', player_ratings)
  print ('After adjustment: ', result)  
  return result

def compose_team(players, Y):
  global blank_rank
  print('Get team rating by individual: ', players, team_X)
  prob_adjusted = 1
  player_ranks = []
  for player in players:
    if player in all_ranking:
      player_rating = all_ranking[player]
    else:
      player_rating = blank_rank
    players_rank.append(player_rating)
    #print('player:',player,'rating:',player_rating)
    prob_adjusted *= (1 - sigmoid(player_rating + Y))
  #print('prob_by_member:', prob_adjusted)
  #print('prob_by_condition', 1-sigmoid(team_X+Y))
  print('Players rank:',player_ranks)
  result = sigm_reverse(1-prob_adjusted) - Y 
  print('Team rank: ', result)
  return result

def get_team_rank(players):
  if len(players)==0:
    return sigm_reverse(1 - (1 - sigmoid(blank_rank)**5))
  
  players_ranks = []
  for player in players:
    if player in all_ranking:
      player_rating = all_ranking[player]
    else:
      player_rating = blank_rank
    player_ranks.append(player_rating)
    player_prob = (1 - sigmoid(player_rating))
    prob *= player_prob
  print ('Players ranks: ',player_ranks)
  result = sigm_reverse(1-prob)
  print('Team rank:') 
  return result

In [0]:
# creating storage for competetion results

logistic = LogisticRegression(fit_intercept = False)

def record_item(item, value):
  all_results_upd[item][1] += 1
  all_results_upd[item][0] += value

def upd_EM(id, update_questions):
  task3 = []
  words = []
   
  for n in range(len(results_cleaned2[id])): # command id
    for p in range(len(results_cleaned2[id][n][0])): #answers
        quest = str(id)+'_'+str(p)
        team = n
        task3.append((quest, team,results_cleaned2[id][n][0][p]))
        words.append(quest)
        words.append(team)

  if len(words)==0:
    return 
  
  words = set(words) # encoding questions and teams no

  idx_to_word = dict(enumerate(words))
  word_to_idx = dict([(y,x) for x,y in idx_to_word.items()])
  #preparing matrix
  X = np.zeros((len(task3),len(words)))
  Y = np.zeros(len(task3))
  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0

  if len(set(Y)) <= 1:
    print ('<2 categories , skipping')
    return

  logistic.fit(X, Y)

  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0


  for i in range(len(task3)):
    question = word_to_idx[task3[i][0]]
    team = word_to_idx[task3[i][1]]
    k_quest = logistic.coef_[0][question]
    k_team = logistic.coef_[0][team]
    players_list = results_cleaned2[id][task3[i][1]][2]
    if (update_questions):
      K = compose_team(players_list, all_ranking[task3[i][0]])
      record_item(task3[i][0], k_quest + k_team - K)
    else: # team members
      #print(players_list, k_team, task3[i][0],all_ranking[task3[i][0]])
      ranking_split = adjust_individual(players_list, k_team,all_ranking[task3[i][0]])
      for j in range(len(players_list)):
        record_item(players_list[j], ranking_split[j])

In [25]:
import tqdm

all_results_upd = {}
for s in all_words:
  all_results_upd[s] = [0,0] # total rating and num of competetions

def record_new_ratings():
  global blank_rank
  global all_results_upd
  for k,v in all_results_upd.items():
    if v[1] != 0:
      all_ranking[k] = v[0]/v[1] 
  print('blank_rank before',blank_rank)
  blank_rank = stat.mean ([ v for k,v in all_ranking.items() if is_player(k)])
  print('blank_rank after',blank_rank)

  all_results_upd = {}
  for s in all_words:
    all_results_upd[s] = [0,0] # total rating and num of competetions


def calc_corellations():
  spearman = []
  pearson = []

  for tourn_id in test_id:
    #print('Tournament:', tourn_id)
    rating = []
    ranking = []
    i=1
    for team in results_cleaned2[tourn_id]:
      rating.append(get_team_rank(team[2]))
      ranking.append(-i)
      i+=1
    #print ('Spearman: ',stats.spearmanr(rating, ranking)[0])
    #print ('Pearson: ',stats.pearsonr(rating,ranking)[0])
    # ставим минус т.к. у нас места возрастают а сила убывает
    spearman.append(stats.spearmanr(rating, ranking)[0])
    pearson.append(stats.pearsonr(rating,ranking)[0])
  
  print('Spearman: ',stat.mean(spearman))
  print('Pearson: ',stat.mean(pearson))  


def EM_cycle():  
  print('Updating individual ranking')
  for tourn_id in tqdm.tqdm(train_id):
    upd_EM(tourn_id, update_questions = False)

  record_new_ratings()
  print('Updating questions')
  for tourn_id in tqdm.tqdm(train_id):
    upd_EM(tourn_id, update_questions = True)

  calc_corellations()

EM_cycle()




  0%|          | 0/675 [00:00<?, ?it/s]

Updating individual ranking





  0%|          | 1/675 [00:00<05:19,  2.11it/s]


  0%|          | 2/675 [00:04<16:54,  1.51s/it]


  0%|          | 3/675 [00:07<23:35,  2.11s/it]


  1%|          | 4/675 [00:11<28:41,  2.57s/it]


  1%|          | 5/675 [00:14<30:29,  2.73s/it]


  1%|          | 7/675 [00:15<22:54,  2.06s/it]


  1%|          | 8/675 [00:16<17:45,  1.60s/it]


  1%|▏         | 9/675 [00:16<14:55,  1.35s/it]


  1%|▏         | 10/675 [00:17<12:45,  1.15s/it]


  2%|▏         | 11/675 [00:18<11:04,  1.00s/it]


  2%|▏         | 12/675 [00:18<09:10,  1.20it/s]


  2%|▏         | 13/675 [00:20<12:38,  1.15s/it]


  2%|▏         | 14/675 [00:25<26:24,  2.40s/it]


  2%|▏         | 15/675 [00:26<19:02,  1.73s/it]


  2%|▏         | 16/675 [00:26<15:09,  1.38s/it]


  3%|▎         | 17/675 [00:27<12:00,  1.10s/it]


  3%|▎         | 18/675 [00:27<09:24,  1.16it/s]


  3%|▎         | 19/675 [00:27<07:44,  1.41it/s]


  3%|▎         | 20/675 [00:27<06:08,  1.78it/s]


  3%|▎         | 21/675 [00:28<05:05

<2 categories , skipping





 99%|█████████▊| 666/675 [04:38<00:00,  9.48it/s]


 99%|█████████▉| 668/675 [04:38<00:00,  9.85it/s]


 99%|█████████▉| 670/675 [04:38<00:00, 10.83it/s]


100%|█████████▉| 672/675 [04:38<00:00,  9.94it/s]


100%|█████████▉| 672/675 [04:57<00:00,  9.94it/s]


100%|█████████▉| 674/675 [05:01<00:03,  3.48s/it]


100%|██████████| 675/675 [05:48<00:00,  1.94it/s]



  0%|          | 0/675 [00:00<?, ?it/s]

blank_rank before -0.5379797641338613
blank_rank after -168.62007587993944
Updating questions





  0%|          | 1/675 [00:00<03:14,  3.47it/s]


  0%|          | 2/675 [00:03<12:29,  1.11s/it]


  0%|          | 3/675 [00:06<17:43,  1.58s/it]


  1%|          | 4/675 [00:08<21:51,  1.96s/it]


  1%|          | 5/675 [00:11<23:23,  2.10s/it]


  1%|          | 7/675 [00:11<17:28,  1.57s/it]


  1%|          | 8/675 [00:12<13:16,  1.19s/it]


  1%|▏         | 9/675 [00:12<10:57,  1.01it/s]


  1%|▏         | 10/675 [00:13<09:13,  1.20it/s]


  2%|▏         | 11/675 [00:13<07:50,  1.41it/s]


  2%|▏         | 12/675 [00:13<06:22,  1.73it/s]


  2%|▏         | 13/675 [00:15<09:10,  1.20it/s]


  2%|▏         | 14/675 [00:19<20:25,  1.85s/it]


  2%|▏         | 16/675 [00:20<14:58,  1.36s/it]


  3%|▎         | 17/675 [00:20<11:18,  1.03s/it]


  3%|▎         | 18/675 [00:20<08:31,  1.29it/s]


  3%|▎         | 19/675 [00:20<06:41,  1.63it/s]


  3%|▎         | 20/675 [00:20<05:05,  2.14it/s]


  3%|▎         | 21/675 [00:20<04:03,  2.69it/s]


  3%|▎         | 22/675 [00:21<03:19

<2 categories , skipping





 99%|█████████▉| 670/675 [03:12<00:00, 18.12it/s]


100%|█████████▉| 673/675 [03:19<00:01,  1.32it/s]


100%|█████████▉| 673/675 [03:38<00:01,  1.32it/s]


100%|██████████| 675/675 [04:11<00:00,  2.69it/s]


UnboundLocalError: ignored

In [29]:


for i in range(10):
  print('итерация ',i)
  EM_cycle()

ValueError: ignored